In [47]:
from IPython.display import HTML
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import precision_score

import urllib2
import bs4
import time
import operator
import re 
    
%matplotlib inline
import seaborn as sns

In [48]:
#Time Series Data
UR = pd.read_csv('UnempRate.csv', sep=',', na_values=[" ", ""])
crime = pd.read_csv('violent_crime_count_age17andup.csv', sep=',', na_values=[" ", ""])
#prescription is per 100
prescription = pd.read_csv('prescription_rate.csv', sep=',', na_values=[" ", ""])
population = pd.read_csv('population.csv', sep=',', na_values=[" ", ""])
birth_rate = pd.read_csv('birth_rate.csv', sep=',', na_values=[" ", ""])
death_rate = pd.read_csv('death_rate.csv', sep=',', na_values=[" ", ""])

In [49]:
#Normalizing Time-Series Data
birth_rate.iloc[:,1:]=birth_rate.iloc[:,1:]/1000
death_rate.iloc[:,1:]=death_rate.iloc[:,1:]/1000
prescription.iloc[:,1:]=prescription.iloc[:,1:]/100

In [57]:
#not normalized
prescription
crime

,County,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Appling,0.889,1.163,1.234,1.069,1.163,1.055,1.303,1.286,1.297,1.295,1.237
1,Atkinson,0.494,0.718,0.666,0.668,0.718,0.714,0.759,0.741,0.713,0.779,0.679
2,Bacon,1.457,1.987,1.691,1.490,1.987,1.658,1.753,1.394,1.457,1.512,1.575
3,Baker,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,Baldwin,0.850,1.165,0.917,0.997,1.165,1.200,1.204,1.182,1.130,1.028,0.984
5,Banks,0.070,0.364,0.208,0.211,0.364,0.391,0.602,0.596,0.511,0.480,0.423
6,Barrow,0.920,0.873,0.949,0.970,0.873,0.913,0.974,0.930,0.923,0.877,0.846
7,Bartow,0.975,1.152,1.121,1.160,1.152,1.177,1.205,1.150,1.121,1.046,1.029
8,Ben Hill,1.333,1.425,1.547,1.438,1.425,1.387,1.798,1.815,1.990,2.081,1.994
9,Berrien,1.001,1.009,0.998,0.977,1.009,0.965,0.958,0.719,0.618,0.573,0.561


In [68]:
#Non-Time Series Data
data = pd.read_csv('cleaned_GaCountiesODDeath.csv', na_values=[" ", ""] ,sep=',', index_col=0).sort('County').reset_index(drop=True)
hospital = pd.read_csv('cleaned_GaHospitals.csv', na_values=[" ", ""] ,sep=',', index_col=0).fillna(0)
dropbox = pd.read_csv('drug_dropbox.csv', na_values=[" ", ""] ,sep=',').fillna(0)
ems = pd.read_csv('cleaned_ems.csv', na_values=[" ", ""] ,sep=',', index_col=0).fillna(0)
rehab = pd.read_csv('cleaned_rehab.csv', na_values=[" ", ""] ,sep=',', index_col=0).fillna(0)

hospital=hospital.rename(columns = {'Count':'hospital_count'})
dropbox=dropbox.rename(columns = {'COUNT':'dropbox_count'})
dropbox=dropbox.rename(columns = {'24Availability':'dropbox_247availability'})

In [69]:
normalized_data = data.iloc[:,1:]/population.iloc[:,1:]

In [70]:
df = data[['County','Rural']].copy()
df1 = pd.concat([df, rehab['Rehab_Count'],  ems['EMS_COUNT'],
                 hospital['hospital_count'], dropbox['dropbox_247availability'], dropbox['dropbox_count']], axis=1)

In [73]:
#data[['Counties','Rural', '2016', '1yrlag', '2yrlag', '10yrlag']]
df1['1yrlag']=(data['2016']-data['2015'])/data['2015']
df1['5yrlag']=(data['2016']-data['2011'])/data['2011']
df1['10yrlag']=(data['2016']-data['2006'])/data['2006']

df1['UR_1yrlag']=(UR['2016']-UR['2015'])/UR['2015']
df1['UR_5yrlag']=(UR['2016']-UR['2011'])/UR['2011']
df1['UR_10yrlag']=(UR['2016']-UR['2006'])/UR['2006']

df1['CR_1yrlag']=(crime['2015']-crime['2014'])/crime['2014']
df1['CR_5yrlag']=(crime['2015']-crime['2010'])/crime['2010']
df1['CR_10yrlag']=(crime['2015']-crime['2005'])/crime['2005']

df1['PR_1yrlag']=(prescription['2016']-prescription['2015'])/prescription['2015']
df1['PR_5yrlag']=(prescription['2016']-prescription['2011'])/prescription['2011']
df1['PR_10yrlag']=(prescription['2016']-prescription['2006'])/prescription['2006']

df1['BR_1yrlag']=(birth_rate['2016']-birth_rate['2015'])/birth_rate['2015']
df1['BR_5yrlag']=(birth_rate['2016']-birth_rate['2011'])/birth_rate['2011']
df1['BR_10yrlag']=(birth_rate['2016']-birth_rate['2006'])/birth_rate['2006']

df1['DR_1yrlag']=(death_rate['2016']-death_rate['2015'])/death_rate['2015']
df1['DR_5yrlag']=(death_rate['2016']-death_rate['2011'])/death_rate['2011']
df1['DR_10yrlag']=(death_rate['2016']-death_rate['2006'])/death_rate['2006']

df1=df1.replace([np.inf, -np.inf], np.nan)
df1 = df1.fillna(0)

death_rate_2016=death_rate.rename(columns = {'2016':'2016_death_rate'})
birth_rate_2016=birth_rate.rename(columns = {'2016':'2016_birth_rate'})
prescription_2016=prescription.rename(columns = {'2016':'2016_prescription'})

df2 = pd.concat([df1, death_rate_2016['2016_death_rate'], birth_rate_2016['2016_birth_rate'], prescription_2016['2016_prescription'], normalized_data['2016']], axis=1)
df2=df2.rename(columns = {'2016':'2016_ODRate'})

In [74]:
df2.columns

Index([u'County', u'Rural', u'Rehab_Count', u'EMS_COUNT', u'hospital_count',
       u'dropbox_247availability', u'dropbox_count', u'1yrlag', u'5yrlag',
       u'10yrlag', u'UR_1yrlag', u'UR_5yrlag', u'UR_10yrlag', u'CR_1yrlag',
       u'CR_5yrlag', u'CR_10yrlag', u'PR_1yrlag', u'PR_5yrlag', u'PR_10yrlag',
       u'BR_1yrlag', u'BR_5yrlag', u'BR_10yrlag', u'DR_1yrlag', u'DR_5yrlag',
       u'DR_10yrlag', u'County', u'2000', u'2001', u'2002', u'2003', u'2004',
       u'2005', u'2006', u'2007', u'2008', u'2009', u'2010', u'2011', u'2012',
       u'2013', u'2014', u'2015', u'2016_death_rate', u'County', u'2000',
       u'2001', u'2002', u'2003', u'2004', u'2005', u'2006', u'2007', u'2008',
       u'2009', u'2010', u'2011', u'2012', u'2013', u'2014', u'2015',
       u'2016_birth_rate', u'County', u'2006', u'2007', u'2008', u'2009',
       u'2010', u'2011', u'2012', u'2013', u'2014', u'2015',
       u'2016_prescription', u'2016_ODRate'],
      dtype='object')

In [24]:
#52,404/323100000
#2015
nat_OD_rate=0.00016219127
dangerous = df2[df2['2016_ODRate']>nat_OD_rate]
ok = df2[df2['2016_ODRate']<=nat_OD_rate]

In [56]:
df2

,County,Rural,Rehab_Count,2016_ODRate,2016_ODRate,2016_ODRate,EMS_COUNT,hospital_count,dropbox_247availability,dropbox_count,...,PR_1yrlag,PR_5yrlag,PR_10yrlag,BR_1yrlag,BR_5yrlag,BR_10yrlag,DR_1yrlag,DR_5yrlag,DR_10yrlag,2016_ODRate
0,Appling,1,0,1.237,0.013563,0.011610,1,0,1,1,...,-0.044788,0.172512,0.391451,-0.037731,-0.040757,-0.032132,0.024701,0.100460,-0.007407,0.000163
1,Atkinson,1,0,0.679,0.017927,0.007339,0,0,0,0,...,-0.128370,-0.049020,0.374494,0.177560,0.094003,-0.155060,0.320503,-0.070309,-0.339263,0.000121
2,Bacon,1,0,1.575,0.012714,0.009623,1,0,0,1,...,0.041667,-0.050060,0.080988,-0.148873,0.033370,-0.175538,-0.016046,-0.212045,-0.106060,0.000000
3,Baker,1,0,0.000,0.010143,0.008558,1,0,0,1,...,0.000000,0.000000,0.000000,-0.095897,-0.021526,0.292040,0.017116,0.070206,0.165342,0.000000
4,Baldwin,0,1,0.984,0.009760,0.009981,1,2,1,1,...,-0.042802,-0.180000,0.157647,-0.018961,-0.111911,-0.168930,0.081872,0.224818,0.135277,0.000000
5,Banks,1,0,0.423,0.011718,0.008355,1,0,0,1,...,-0.118750,0.081841,5.042857,-0.021191,0.189421,-0.114650,-0.096894,0.136966,-0.030297,0.000217
6,Barrow,0,1,0.846,0.014110,0.007725,1,1,0,2,...,-0.035348,-0.073384,-0.080435,-0.003088,-0.082432,-0.219392,0.047608,0.046655,0.178995,0.000169
7,Bartow,0,1,1.029,0.012581,0.008549,1,1,1,2,...,-0.016252,-0.125743,0.055385,-0.038482,-0.014972,-0.251839,-0.008083,0.028507,0.125486,0.000202
8,Ben Hill,1,0,1.994,0.013643,0.012949,1,1,1,1,...,-0.041807,0.437635,0.495874,-0.027279,-0.083199,-0.286650,0.010204,0.096098,0.176910,0.000058
9,Berrien,1,1,0.561,0.013857,0.011381,1,1,0,1,...,-0.020942,-0.418653,-0.439560,-0.009115,0.197646,-0.084032,0.096147,0.085371,0.148933,0.000053


In [169]:
features = list(df2.columns[1:-1])
X = df2[features]
Y = df2['2016_ODRate']

In [170]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [171]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, Y_train)

In [172]:
Y_pred = clf.predict(X_test)

In [8]:
df2.to_csv('AggredgatedData.csv')

In [177]:
Y_test

78     0.000000
155    0.001256
128    0.000197
55     0.000018
94     0.000072
29     0.000000
147    0.000839
51     0.000157
98     0.000000
141    0.000747
19     0.000019
60     0.000135
15     0.000013
65     0.000235
24     0.000017
30     0.000007
126    0.000350
101    0.000037
96     0.000173
16     0.000000
151    0.004232
18     0.000158
12     0.000218
9      0.000053
31     0.000146
125    0.001173
95     0.000149
56     0.000083
145    0.000250
152    0.000376
135    0.001879
76     0.000000
75     0.000007
138    0.003248
2      0.000000
86     0.000000
45     0.000073
42     0.000075
68     0.000000
118    0.001027
26     0.000161
137    0.001103
146    0.000155
90     0.000000
66     0.000000
36     0.000000
82     0.000526
22     0.000015
85     0.000000
81     0.000000
112    0.000261
11     0.000308
109    0.000083
Name: 2016_ODRate, dtype: float64